# 0. Setup

Install required packages

In [ ]:
!pip install --upgrade pip
!pip install transformers accelerate bitsandbytes
!pip install datasets pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 101.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Set Up API Key (If you don't have llama 3.1 key text file, run this code and delete code using that text file)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

Import Libraries and Set Up API Key

In [ ]:
import os
import torch
import transformers
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 파일에서 API 키 읽기
with open('/content/drive/My Drive/2024_02_ML/llama_3.1_key.txt', 'r') as file:
    os.environ['HF_TOKEN'] = file.read().strip()

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

Mounted at /content/drive


In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a chatbot"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][-1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': "I'm an artificial intelligence (AI) chatbot designed to assist and communicate with users in a helpful and informative way. I'm a machine learning model trained on a vast amount of text data, which enables me to understand and respond to a wide range of questions and topics.\n\nI don't have personal experiences, emotions, or consciousness like humans do. My purpose is to provide accurate and up-to-date information, answer questions, and engage in conversation to the best of my abilities.\n\nI'm constantly learning and improving, so I appreciate any feedback or corrections you may have. I'm here to help with any questions or topics you'd like to discuss, so feel free to ask me anything!"}


# 1. Test in basic LLAMA-3.1

MedQA Dataset

1. Check available answers

In [ ]:
# MedQA 데이터셋 로드
from datasets import load_dataset

medqa = load_dataset("pubmed_qa", "pqa_labeled")

# 가능한 답안 목록
valid_answers = {"yes", "no", "maybe"}

# 데이터셋의 모든 정답 확인
def check_answers(dataset):
    invalid_answers = []  # 유효하지 않은 답안 수집
    for idx, sample in enumerate(dataset["train"]):  # train 데이터만 순회
        answer = sample["final_decision"].strip().lower()
        if answer not in valid_answers:
            invalid_answers.append((idx, answer))  # 인덱스와 함께 저장

    if not invalid_answers:
        print("All answers are valid (yes, no, maybe).")
    else:
        print(f"Invalid answers found: {len(invalid_answers)}")
        for idx, invalid in invalid_answers[:5]:  # 첫 5개의 에러 출력
            print(f"Index: {idx}, Answer: {invalid}")

# MedQA train split에서 답안 확인
check_answers(medqa)


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

All answers are valid (yes, no, maybe).


In [ ]:
from datasets import load_dataset

# PubMedQA 데이터셋 로드
medqa = load_dataset("pubmed_qa", "pqa_labeled")

# Train 데이터를 분리하여 Test 데이터 생성
split_data = medqa["train"].train_test_split(test_size=0.1, seed=42)
train_data = split_data["train"]
test_data = split_data["test"]

# 분리된 데이터셋 확인
print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# 첫 번째 샘플 확인
print(test_data[0])


Train samples: 900
Test samples: 100
{'pubid': 15483019, 'question': 'Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?', 'context': {'contexts': ['To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer.', 'Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of cisplatin monotherapy after cystectomy. This study had a negative result. The outcome of these 35 patients (protocol group) was compared with an age- and tumor-stage-matched cohort (matched group; n = 35) who also underwent cystectomy during the same period, but were not entered into the SAKK study, as well as the remaining 57 patients treated d

2. Test basic LLAMA 3.1 for MedQA dataset

In [ ]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer

# LLAMA 모델 로드
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id  # pad_token_id 설정

llama_pipeline = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},  # float16으로 설정
    device_map="auto",
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# MedQA 데이터셋 로드
medqa = load_dataset("pubmed_qa", "pqa_labeled")

# Train 데이터를 분리하여 Test 데이터 생성
split_data = medqa["train"].train_test_split(test_size=0.1, seed=42)
train_data = split_data["train"]
test_data = split_data["test"]

print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# MedQA 테스트 데이터 가공
def preprocess_medqa(data):
    processed_data = []
    for item in data:
        # 질문과 선택지 포함한 프롬프트 생성
        question = item["question"]
        prompt = (
            f"Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.\n\n"
            f"Question: {question}\n\n"
            f"Answer (only Yes, No, or Maybe):"
        )
        processed_data.append({"prompt": prompt, "id": item["pubid"], "label": item.get("final_decision", None)})
    return processed_data

test_prompts = preprocess_medqa(test_data)

# 응답 추출 함수
def extract_answer(prompt, generated_text):
    # 프롬프트 길이를 계산하고 이후 부분만 반환
    return generated_text[len(prompt):].strip()

# 모델 실행 및 예측
def generate_medqa_answers(prompts, pipeline, choices=["Yes", "No", "Maybe"]):
    results = []
    for idx, sample in enumerate(prompts):
        # 모델 예측
        output = pipeline(sample["prompt"], max_new_tokens=5, do_sample=True, top_p=0.9, temperature=0.7)
        generated_answer = output[0]["generated_text"].strip()

        # 응답에서 프롬프트 제거
        clean_answer = extract_answer(sample["prompt"], generated_answer)

        # 선택지 필터링
        filtered_answer = "Maybe"
        for choice in choices:
            if choice.lower() in clean_answer.lower():
                filtered_answer = choice
                break

        # 결과 저장
        result = {
            "id": sample["id"],
            "question": sample["prompt"],
            "predicted_answer": filtered_answer,
            "true_label": sample["label"]
        }
        results.append(result)

        # 각 결과 출력
        print(f"Sample {idx+1}:")
        print(f"ID: {result['id']}")
        print(f"Question: {result['question']}")
        print(f"Predicted Answer: {result['predicted_answer']}")
        print(f"True Label: {result['true_label']}\n")

    return results

# 답안 생성
predictions = generate_medqa_answers(test_prompts, llama_pipeline)

# 최종 성능 평가 (정답 라벨이 있는 경우)
correct = sum(1 for r in predictions if r["true_label"] and r["predicted_answer"].lower() == r["true_label"].lower())
accuracy = (correct / len(predictions)) * 100 if len(predictions) > 0 else 0
print(f"Final Accuracy: {accuracy:.2f}%")


Train samples: 900
Test samples: 100
Sample 1:
ID: 15483019
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?

Answer (only Yes, No, or Maybe):
Predicted Answer: Yes
True Label: yes

Sample 2:
ID: 24450673
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Delayed imaging in routine CT examinations of the abdomen and pelvis: is it worth the additional cost of radiation and time?

Answer (only Yes, No, or Maybe):
Predicted Answer: Maybe
True Label: no

Sample 3:
ID: 9582182
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Does the SCL 90-R obsessive-compulsive dimension identify cognitive impairments?

Answer (only Yes, No, or Maybe):
Predicted Answer: No
Tr

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Sample 10:
ID: 23002947
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Does feeding tube insertion and its timing improve survival?

Answer (only Yes, No, or Maybe):
Predicted Answer: Yes
True Label: no

Sample 11:
ID: 12970636
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Does early discharge with nurse home visits affect adequacy of newborn metabolic screening?

Answer (only Yes, No, or Maybe):
Predicted Answer: No
True Label: no

Sample 12:
ID: 23719685
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Maybe.

Question: Does high-dose radiotherapy benefit palliative lung cancer patients?

Answer (only Yes, No, or Maybe):
Predicted Answer: No
True Label: no

Sample 13:
ID: 10340286
Question: Please answer the following medical question strictly with one of these options: Yes, No, or Ma

In [ ]:
# MedQA 테스트 데이터 가공 (context 포함)
def preprocess_medqa_with_context(data):
    processed_data = []
    for item in data:
        # 질문과 컨텍스트 포함한 프롬프트 생성
        question = item["question"]
        context = " ".join(item["context"]["contexts"])  # 여러 context를 결합
        prompt = (
            f"Answer the following medical question based on the provided context strictly with one of these options: Yes, No, or Maybe.\n\n"
            f"Context: {context}\n\n"
            f"Question: {question}\n\n"
            f"Answer (only Yes, No, or Maybe):"
        )
        processed_data.append({"prompt": prompt, "id": item["pubid"], "label": item.get("final_decision", None)})
    return processed_data

test_prompts_with_context = preprocess_medqa_with_context(test_data)

# 응답 추출 함수
def extract_answer(prompt, generated_text):
    # 프롬프트 길이를 계산하고 이후 부분만 반환
    return generated_text[len(prompt):].strip()

# 모델 실행 및 예측
def generate_medqa_answers_with_context(prompts, pipeline, choices=["Yes", "No", "Maybe"]):
    results = []
    for idx, sample in enumerate(prompts):
        # 모델 예측
        output = pipeline(sample["prompt"], max_new_tokens=2, do_sample=True, top_p=0.9, temperature=0.01)
        generated_answer = output[0]["generated_text"].strip()

        # 응답에서 프롬프트 제거
        clean_answer = extract_answer(sample["prompt"], generated_answer)

        # 선택지 필터링
        filtered_answer = "Maybe"
        for choice in choices:
            if choice.lower() in clean_answer.lower():
                filtered_answer = choice
                break

        # 결과 저장
        result = {
            "id": sample["id"],
            "question": sample["prompt"],
            "predicted_answer": filtered_answer,
            "true_label": sample["label"]
        }
        results.append(result)

        # 각 결과 출력
        print(f"Sample {idx+1}:")
        print(f"ID: {result['id']}")
        print(f"Question: {result['question']}")
        print(f"Predicted Answer: {result['predicted_answer']}")
        print(f"True Label: {result['true_label']}\n")

    return results

# 답안 생성
predictions_with_context = generate_medqa_answers_with_context(test_prompts_with_context, llama_pipeline)

# 최종 성능 평가 (정답 라벨이 있는 경우)
correct = sum(1 for r in predictions_with_context if r["true_label"] and r["predicted_answer"].lower() == r["true_label"].lower())
accuracy = (correct / len(predictions_with_context)) * 100 if len(predictions_with_context) > 0 else 0
print(f"Final Accuracy with Context: {accuracy:.2f}%")

Sample 1:
ID: 15483019
Question: Answer the following medical question based on the provided context strictly with one of these options: Yes, No, or Maybe.

Context: To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer. Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of cisplatin monotherapy after cystectomy. This study had a negative result. The outcome of these 35 patients (protocol group) was compared with an age- and tumor-stage-matched cohort (matched group; n = 35) who also underwent cystectomy during the same period, but were not entered into the SAKK study, as well as the remaining 57 patients treated during the study period for the same indication (remaining

3. Test LLAMA 3.1 with RAG for MedQA dataset

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Sentence-BERT 모델 로드
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # 작은 크기의 빠른 모델

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. TF-IDF 기반 검색 준비
# Train 데이터에서 질문과 답변 결합

# train_questions, train_contexts, train_answers는 데이터셋 로드 이후 미리 정의된 변수로 가정
# 예: train_questions = ["Question 1", "Question 2", ...], train_contexts = ["Context 1", ...], train_answers = ["Yes", ...]
train_questions = [item["question"] for item in train_data]
train_contexts = [" ".join(item["context"]["contexts"]) for item in train_data]
train_answers = [item["final_decision"] for item in train_data]

train_combined = [f"{q} {c}" for q, c in zip(train_questions, train_contexts)]
vectorizer = TfidfVectorizer(stop_words="english")
train_vectors = vectorizer.fit_transform(train_combined)
train_embeddings = embedding_model.encode(train_combined, convert_to_tensor=True)

# 2. 유사한 질문 검색
def retrieve_similar_questions(query_question, query_context, top_k=5):
    query_combined = f"{query_question} {query_context}"
    query_embedding = embedding_model.encode(query_combined, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, train_embeddings).squeeze()
    top_indices = torch.topk(similarities, k=top_k).indices.tolist()
    return [(train_questions[i], train_contexts[i], train_answers[i]) for i in top_indices]

# 3. 테스트 데이터 가공 및 프롬프트 생성
def preprocess_medqa_with_retrieval(data, top_k=3):
    processed_data = []
    for item in data:
        question = item["question"]
        context = " ".join(item["context"]["contexts"])

        # 질문 + 컨텍스트 기반 유사 문제 검색
        retrieved_data = retrieve_similar_questions(question, context, top_k=top_k)
        retrieved_text = "\n".join(
            [f"Similar Question: {q}\nContext: {c}\nAnswer: {a}" for q, c, a in retrieved_data]
        )

        # 프롬프트 생성
        prompt = (
            f"Answer the following medical question based on the provided context and similar examples. "
            f"Strictly answer with one of these options: Yes, No, or Maybe.\n\n"
            f"Original Context: {context}\n\n"
            f"Relevant Examples:\n{retrieved_text}\n\n"
            f"Question: {question}\n\n"
            f"Answer (only Yes, No, or Maybe):"
        )

        processed_data.append({"prompt": prompt, "id": item["pubid"], "label": item.get("final_decision", None)})
    return processed_data

# 4. 모델 실행 및 예측
def extract_answer(prompt, generated_text):
    """
    프롬프트 길이만큼 제거하여 모델의 순수 답변만 추출
    """
    return generated_text[len(prompt):].strip()

def generate_rag_answers(prompts, llama_pipeline, choices=["Yes", "No", "Maybe"]):
    results = []
    for idx, sample in enumerate(prompts):
        # 모델 예측
        output = llama_pipeline(sample["prompt"], max_new_tokens=2, do_sample=True, top_p=0.9, temperature=0.01)
        generated_text = output[0]["generated_text"].strip()

        # 프롬프트 길이 제거
        clean_answer = extract_answer(sample["prompt"], generated_text)

        # 선택지 필터링
        filtered_answer = "Maybe"
        for choice in choices:
            if choice.lower() in clean_answer.lower():
                filtered_answer = choice
                break

        # 결과 저장
        result = {
            "id": sample["id"],
            "question": sample["prompt"],
            "predicted_answer": filtered_answer,
            "true_label": sample["label"]
        }
        results.append(result)

        # 각 결과 출력
        print(f"Sample {idx+1}:")
        print(f"ID: {result['id']}")
        print(f"Question: {result['question']}")
        print(f"Cleaned Answer: {clean_answer}")
        print(f"Predicted Answer: {result['predicted_answer']}")
        print(f"True Label: {result['true_label']}\n")

    return results


# 5. 실행 예시
test_prompts_with_retrieval = preprocess_medqa_with_retrieval(test_data)
predictions_with_retrieval = generate_rag_answers(test_prompts_with_retrieval, llama_pipeline)

# 6. 성능 평가
correct = sum(1 for r in predictions_with_retrieval if r["true_label"] and r["predicted_answer"].lower() == r["true_label"].lower())
accuracy = (correct / len(predictions_with_retrieval)) * 100 if len(predictions_with_retrieval) > 0 else 0
print(f"Final Accuracy with RAG and Context: {accuracy:.2f}%")


Sample 1:
ID: 15483019
Question: Answer the following medical question based on the provided context and similar examples. Strictly answer with one of these options: Yes, No, or Maybe.

Original Context: To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer. Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of cisplatin monotherapy after cystectomy. This study had a negative result. The outcome of these 35 patients (protocol group) was compared with an age- and tumor-stage-matched cohort (matched group; n = 35) who also underwent cystectomy during the same period, but were not entered into the SAKK study, as well as the remaining 57 patients treated during the study per

4. Test LLAMA 3.1 with optimized prompt RAG for MedQA dataset

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 학습 가능한 프롬프트 클래스 정의
class LearnablePrompt(nn.Module):
    def __init__(self, embedding_dim, num_prompt_tokens):
        super(LearnablePrompt, self).__init__()
        self.prompt_embedding = nn.Parameter(torch.randn(num_prompt_tokens, embedding_dim))

    def forward(self):
        return self.prompt_embedding

# TF-IDF 기반 검색 함수 정의
def retrieve_similar_contexts_tfidf(question, train_data, top_k=3):
    """
    TF-IDF 기반으로 질문과 유사한 컨텍스트를 검색
    """
    train_contexts = [" ".join(item["context"]["contexts"]) for item in train_data]
    vectorizer = TfidfVectorizer()

    # TF-IDF 벡터 생성
    tfidf_matrix = vectorizer.fit_transform([question] + train_contexts)  # 첫 번째는 질문 벡터
    question_vec = tfidf_matrix[0]
    train_vecs = tfidf_matrix[1:]

    # 코사인 유사도 계산
    similarities = cosine_similarity(question_vec, train_vecs).flatten()
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    # NumPy 타입을 Python int로 변환
    top_indices = [int(idx) for idx in top_indices]

    return [train_data[idx] for idx in top_indices]


# 학습 루프 정의
def train_prompt(train_data, learnable_prompt, optimizer, epochs=3):
    learnable_prompt.train()
    for epoch in range(epochs):
        total_loss = 0
        for item in train_data:
            question = item["question"]
            label = item["final_decision"]  # 정답 (Yes, No, Maybe)

            # 손실 계산: 예제에서는 간단한 예시로 학습
            prompt_output = learnable_prompt()
            loss = ((prompt_output.sum() - len(question)) ** 2).mean()  # 손실 함수 예시
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_data):.4f}")

# 프롬프트 길이 잘라내기 함수
def trim_prompt_from_output(prompt, output_text):
    """
    프롬프트 길이만큼 출력에서 제거하여 실제 답변만 추출.
    """
    if output_text.startswith(prompt):
        return output_text[len(prompt):].strip()
    return output_text.strip()

# 테스트 함수 정의
def test_with_rag_and_learned_prompt_tfidf(test_data, train_data, learnable_prompt, pipeline_model, top_k=3, choices=["Yes", "No", "Maybe"]):
    learnable_prompt.eval()
    prompt_embedding = learnable_prompt().detach().cpu().numpy()
    results = []

    for item in test_data:
        question = item["question"]
        context = " ".join(item["context"]["contexts"])  # 컨텍스트 병합

        # TF-IDF 기반 유사 컨텍스트 검색
        retrieved_data = retrieve_similar_contexts_tfidf(question, train_data, top_k)
        retrieved_context = "\n".join([" ".join(r["context"]["contexts"]) for r in retrieved_data])

        # 프롬프트 생성
        prompt = (
            f"Answer the following medical question based on the provided context and similar examples.\n"
            f"Strictly answer with one of these options: Yes, No, or Maybe.\n\n"
            f"Optimized Prompt: {prompt_embedding}\n"
            f"Question: {question}\n"
            f"Retrieved Context: {retrieved_context}\n"
            f"Original Context: {context}\n"
            f"Answer (Yes, No, Maybe):"
        )
        output = pipeline_model(prompt, max_new_tokens=2, do_sample=True, top_p=0.9, temperature=0.01)
        generated_text = output[0]["generated_text"].strip()

        # 프롬프트 길이만큼 잘라내기
        trimmed_answer = trim_prompt_from_output(prompt, generated_text)
        true_label = item["final_decision"]

        # 선택지 필터링
        filtered_answer = "Maybe"
        for choice in choices:
            if choice.lower() in trimmed_answer.lower():
                filtered_answer = choice
                break

        results.append({
            "question": question,
            "predicted_answer": filtered_answer,
            "true_label": true_label
        })

        print(f"Question: {prompt}")
        print(f"Cleaned Answer: {trimmed_answer}")
        print(f"Predicted Answer: {filtered_answer}")
        print(f"True Label: {true_label}\n")

    return results

# 성능 평가 함수
def evaluate_accuracy(predictions):
    correct = sum(1 for r in predictions if r["predicted_answer"].lower() == r["true_label"].lower())
    accuracy = (correct / len(predictions)) * 100 if predictions else 0
    return accuracy

# 프롬프트 초기화
embedding_dim = 4096  # LLAMA 모델의 임베딩 크기
num_prompt_tokens = 10
learnable_prompt = LearnablePrompt(embedding_dim, num_prompt_tokens).to("cuda")

# 옵티마이저 정의
optimizer = Adam(learnable_prompt.parameters(), lr=1e-4)

# 학습
train_prompt(train_data, learnable_prompt, optimizer, epochs=3)

# 테스트
results = test_with_rag_and_learned_prompt_tfidf(test_data, train_data, learnable_prompt, llama_pipeline)

# 성능 평가
accuracy = evaluate_accuracy(results)
print(f"Final Accuracy: {accuracy:.2f}%")


Epoch 1, Loss: 932.4641
Epoch 2, Loss: 901.0509
Epoch 3, Loss: 901.1449
Question: Answer the following medical question based on the provided context and similar examples.
Strictly answer with one of these options: Yes, No, or Maybe.

Optimized Prompt: [[ 0.09774085 -1.5275191  -2.167489   ... -0.23168685  2.0273745
   1.0440289 ]
 [ 0.2327546  -0.06149066 -0.2603709  ...  0.38312042  1.9782028
   0.14129162]
 [ 0.94230205  1.846156    0.78218    ... -1.5313058  -0.10379718
  -0.78793156]
 ...
 [ 0.40559956  0.48224646  0.62486076 ... -1.0657297  -0.84780663
   0.04712997]
 [ 1.2312356   0.10817317 -0.7272326  ...  0.18357924 -1.0972362
  -0.00745147]
 [ 1.7795901  -0.54983836  1.3560647  ... -1.6457759   0.3766925
   1.1022227 ]]
Question: Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?
Retrieved Context: • Robot-assisted radical cystectomy (RARC) remains controversial in terms of oncologic outcomes, especially 